# Overlay fotos

## Introducción

Ahora que tenemos controladas las cuestiones de conversión entre posiciones de píxeles en las fotos y coordenadas terrestres, vamos a enfocarnos a utilizar las librerías que nos permitirán manipular de forma automatizada las fotos y no con editores como Gimp que es lo que hemos utilizado hasta ahora para generar algunas de las representaciones mostradas en los anteriores cuadernos.

Vamos a utilizar la potente librería [OpenCV](https://docs.opencv.org/4.x/index.html).

## Sede Atlantes

Vamos a empezar haciendo un sencillo ejercicio. Estimamos que la ciudad de Zaragoza, sede del equipo Atlantes, aparece entre las fotos 494 y 496 aunque, como la zona está cubierta por las nubes, no podemos señalar visualmente la posición con precisión, pero sí matemáticamente utilizando nuestras nuevas habilidades de cálculo.

Lo que queremos lograr es una pequeña animación con los 3 fotogramas en los que aparezca señalada la ciudad de Zaragoza en su posición correcta en cada uno de los fotogramas. Partimos de las fotos ya rotadas (que almacenamos en el directorio `atlantes_2021-2022_rotadas` al final del primer cuaderno) y de la estructura de datos JSON donde almacenamos los ángulos α de todas ellas (también almacenado en el fichero `alphas_445-622.json` al final del primer cuaderno). La secuencia de trabajo será más o menos la siguiente:

1. Para cada fotograma:
    1. Cargar con OpenCV el fotograma.
    2. Convertir las coordenadas de la ciudad de Zaragoza a posición de pixel del fotograma.
    3. Colocar una marca centrada en el pixel calculado en el paso anterior.
    4. Añadir el fotograma resultante a la animación

In [8]:
import os
import cv2
import json
import pandas
from geopy import distance, point

# Constantes
FOTO_W = 3280
FOTO_H = 2464
S_W = 6.287
F = 5
PHOTO_ROT_DIR_PATH = "atlantes_2021-2022_rotadas"
ATLANTES_FILE = "atlantes_2021-2022.csv"
ALPHAS_FILE = "alphas_445-622.json"
LOGO_FILE = "2018_logo-circulo_100px.png"
ANIM_FILE = "anim.avi"
PHOTO_START = 494
PHOTO_END = 496
RESULT_WIDTH = 640
RESULT_HEIGHT = 480
FPS = 3
OUTPUT = True    # True: Salida a fichero; False: Salida a pantalla

pnt_X = point.Point(41.6516859, -0.9300003) # Zaragoza


# Calcula el pixel (X,Y) correspondiente a unas coordenadas (latitud,longitud) de una foto cuyo pixel central tiene
# unas coordenadas conocidas.
# Argumentos:
#     * pnt_I: Punto (clase geopy.point.Point) medido por AstroPi durante la captura de la foto como posición
#              de la ISS.
#     * alt:   Altitud de la ISS durante la captura de la foto.
#     * alpha: Ángulo de la órbita respecto del ecuador durante la captura de la foto.
#     * pnt_X: Punto (clase geopy.point.Point) del que queremos obtener su posición en la foto.
# Resutado:
#     Tupla (X,Y) con la posición del pixel correspondiente a las coordenadas del punto pnt_X.
def coords2pixel(pnt_I, alpha, alt, pnt_X):
    pnt_O = distance.great_circle(meters=52329).destination(pnt_I, bearing=-61.69-alpha)
    pnt_O.altitude = alt
    
    d_w = S_W * pnt_O.altitude / F
    d_h = d_w * FOTO_H / FOTO_W
    pnt_T = distance.great_circle(meters=d_h/2).destination(pnt_O, bearing=0)
    pnt_B = distance.great_circle(meters=d_h/2).destination(pnt_O, bearing=180)
    pnt_L = distance.great_circle(meters=d_w/2).destination(pnt_O, bearing=270)
    pnt_R = distance.great_circle(meters=d_w/2).destination(pnt_O, bearing=90)
    t_lat, t_lon = pnt_T.latitude, pnt_T.longitude
    b_lat, b_lon = pnt_B.latitude, pnt_B.longitude
    l_lat, l_lon = pnt_L.latitude, pnt_L.longitude
    r_lat, r_lon = pnt_R.latitude, pnt_R.longitude
    x_lat, x_lon = pnt_X.latitude, pnt_X.longitude
    
    # Normalizamos las longitudes para que se puedan hacer los cálculos proporcionales
    if l_lon * r_lon < 0:
        if l_lon < -90:
            l_lon += 360
        if r_lon < -90:
            r_lon += 360
        if x_lon < -90:
            x_lon += 360
    
    if not (b_lat <= x_lat <= t_lat) or not (l_lon <= x_lon <= r_lon):
        raise Exception(f"Las coordenadas ({pnt_X.latitude},{x_lon}) quedan fuera de la foto.")

    X = FOTO_W * ( 1 - (r_lon - x_lon) / (r_lon - l_lon))
    Y = FOTO_H * (t_lat - x_lat) / (t_lat - b_lat)
    return (round(X), round(Y))


# https://stackoverflow.com/questions/70578600/how-to-merge-one-rgba-and-one-rgb-images-in-opencv
def alphaMerge(small_foreground, background, x, y):
    """
    Puts a small BGRA picture in front of a larger BGR background.
    :param small_foreground: The overlay image. Must have 4 channels.
    :param background: The background. Must have 3 channels.
    :param x: X position where to put the overlay's center.
    :param y: Y position where to put the overlay's center.
    :return: a copy of the background with the overlay added.
    """
    result = background.copy()
    # From everything I read so far, it seems we need the alpha channel separately
    # so let's split the overlay image into its individual channels
    fg_b, fg_g, fg_r, fg_a = cv2.split(small_foreground)
    # Make the range 0...1 instead of 0...255
    fg_a = fg_a / 255.0
    # Multiply the RGB channels with the alpha channel
    label_rgb = cv2.merge([fg_b * fg_a, fg_g * fg_a, fg_r * fg_a])
    # Work on a part of the background only
    height, width = small_foreground.shape[0], small_foreground.shape[1]
    mid_h = int(height/2)
    mid_w = int(width/2)
    part_of_bg = result[y-mid_h:y+mid_h, x-mid_w:x+mid_w, :]
    # Same procedure as before: split the individual channels
    bg_b, bg_g, bg_r = cv2.split(part_of_bg)
    # Merge them back with opposite of the alpha channel
    part_of_bg = cv2.merge([bg_b * (1 - fg_a), bg_g * (1 - fg_a), bg_r * (1 - fg_a)])
    # Add the label and the part of the background
    cv2.add(label_rgb, part_of_bg, part_of_bg)
    # Replace a part of the background
    result[y-mid_h:y+mid_h, x-mid_w:x+mid_w, :] = part_of_bg
    return result


path = os.path.dirname(os.path.realpath("__file__"))
photo_rot_dir_path = os.path.join(path, PHOTO_ROT_DIR_PATH)
atlantes_file_path = os.path.join(path, ATLANTES_FILE)
alphas_file_path = os.path.join(path, ALPHAS_FILE)
logo_file_path = os.path.join(path, LOGO_FILE)

alphas_file = open(alphas_file_path)
alphas_json = json.load(alphas_file)

atlantes_data = pandas.read_csv(atlantes_file_path)

logo_cv = cv2.imread(logo_file_path, cv2.IMREAD_UNCHANGED)

if OUTPUT:
    fourcc = cv2.VideoWriter_fourcc(*'MP42')
    anim_file_path = os.path.join(path, ANIM_FILE)
    video = cv2.VideoWriter(anim_file_path, fourcc, float(FPS), (RESULT_WIDTH, RESULT_HEIGHT))

for i in range(PHOTO_START, PHOTO_END+1):
    pnt_I = point.Point(atlantes_data.latitude[i-1], atlantes_data.longitude[i-1])
    alpha = alphas_json[str(i)]
    altitude = atlantes_data.elevation[i-1]
    photo_path = os.path.join(photo_rot_dir_path, "atlantes_%d.jpg" % (i))
    photo_cv = cv2.imread(photo_path)
    try:
        (X, Y) = coords2pixel(pnt_I, alpha, altitude, pnt_X)
        photo_cv = alphaMerge(logo_cv, photo_cv, X, Y)
    except:
        pass
    photo_cv = cv2.resize(photo_cv, (RESULT_WIDTH, RESULT_HEIGHT))
    if OUTPUT:
        video.write(photo_cv)
    else:
        cv2.imshow("Home of Atlantes", photo_cv)
        cv2.waitKey(int(1000/FPS))

cv2.destroyAllWindows()

alphas_file.close()
video.release()